# Normalize CST files

In [2]:
import numpy as np
import os 
import json  
from pathlib import Path

In [9]:
# Base project folder
base_folder = Path.home() / "fcc_ee_pywit_model" / "CEI_section_meeting"

# Input and output folders relative to base_folder
import_folder = base_folder / "1_CST_results/Collimator_SquarePipe/taper_angle_15deg/Dipolar"
output_folder = base_folder / "3_Impedance_Model/Using_CST_results/3.1_Collimators_and pipe/Dipolar_H15deg_V2.5deg/collimator_normalized/H15_V15"


In [10]:
for filename in os.listdir(import_folder):
    dict_out = {}
    table = np.loadtxt(f'{import_folder}/{filename}', skiprows=3)
    if filename.startswith("Impedance"):
        table[:, 0] *= 1e9
        table[:, 1:3] *= -1e3
        if "dipX" in filename:
            component = "zxdip"
        elif "dipY" in filename:
            component = "zydip"
        else: 
            raise ValueError("Nor dipX either dipY found in filename")
        
        dict_out[component] = {}
        dict_out[component]["frequency"] = table[:, 0].tolist()
        dict_out[component]["real impedance"] = table[:, 1].tolist()
        dict_out[component]["imaginary impedance"] = table[:, 2].tolist()

    elif filename.startswith("Wake"):
        table[:, 0] /= 1e3
        table[:, 1] *= 1e3
        if "dipX" in filename:
            component = "wxdip"
        elif "dipY" in filename:
            component = "wydip"
        else: 
            raise ValueError("Nor dipX either dipY found in filename")
        
        dict_out[component] = {}
        dict_out[component]["distance"] = table[:, 0].tolist()
        dict_out[component]["wake"] = table[:, 1].tolist()
    else:
        raise ValueError("Nor impedance either wake found in filename")
   
    # save the dictionary in a json file
    
    with open(f'{output_folder}/{filename}'.replace("txt", "json"), "w") as f:
        json.dump(dict_out, f, indent=2)
